In [ ]:
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from IPython.display import HTML, display

In [ ]:
RESOURCES_FOLDERNAME = "test_SCENIC/test_run/protocol/resources/"
AUXILLIARIES_FOLDERNAME = "test_SCENIC/test_run/protocol/auxilliaries/"
RESULTS_FOLDERNAME = "test_SCENIC/ImputeEvaluation/results/"

In [ ]:
# Downloaded fromm pySCENIC github repo: https://github.com/aertslab/pySCENIC/tree/master/resources
HUMAN_TFS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'lambert2018.txt')
# Ranking databases. Downloaded from cisTargetDB: https://resources.aertslab.org/cistarget/
RANKING_DBS_FNAMES = list(map(lambda fn: os.path.join(AUXILLIARIES_FOLDERNAME, fn),
                       ['hg19-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings.feather',
                       'hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings.feather',
                        'hg19-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings.feather']))
# Motif annotations. Downloaded from cisTargetDB: https://resources.aertslab.org/cistarget/
MOTIF_ANNOTATIONS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl')

In [ ]:
Selected_Genes = ['PAX5','TCF7','EOMES','TBX21','PRRX2','MAFB','SOX7','MITF','MYC','TWIST1']

In [ ]:
#raw

In [ ]:
DATASET_ID = "GSE115978raw"

In [ ]:

CELL_ANNOTATIONS_FNAME = "test_SCENIC/ImputeEvaluation/GSE115978_cell.annotations.csv"

SAMPLE_METADATA_FNAME = os.path.join(RESOURCES_FOLDERNAME, "mmc1.xlsx")

EXP_MTX_COUNTS_FNAME = "test_SCENIC/ImputeEvaluation/SCENIC_GSE115978_rawlognorm.txt"

In [ ]:
METADATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.metadata.csv'.format(DATASET_ID))
ADJACENCIES_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.adjacencies.tsv'.format(DATASET_ID))
MOTIFS_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.motifs.csv'.format(DATASET_ID))
REGULONS_DAT_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.regulons.dat'.format(DATASET_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.auc.csv'.format(DATASET_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.bin.csv'.format(DATASET_ID))
THR_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.thresholds.csv'.format(DATASET_ID))
ANNDATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.h5ad'.format(DATASET_ID))
EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.forSCENIC.csv'.format(DATASET_ID))

In [ ]:
df_annotations = pd.read_csv(CELL_ANNOTATIONS_FNAME)
df_annotations['samples'] = df_annotations['samples'].str.upper()
df_annotations.rename(columns={'cell.types': 'cell_type', 'cells': 'cell_id', 'samples': 'sample_id', 
                               'treatment.group': 'treatment_group', 'Cohort': 'cohort'}, inplace=True)
df_annotations['cell_type'] = df_annotations.cell_type.replace({
    'Mal': 'Malignant Cell',
    'Endo.': 'Endothelial Cell',
    'T.CD4': 'Thelper Cell',
    'CAF': 'Fibroblast',
    'T.CD8': 'Tcytotoxic Cell',
    'T.cell': 'T Cell',
    'NK': 'NK Cell',
    'B.cell': 'B Cell'})
df_samples = pd.read_excel(SAMPLE_METADATA_FNAME, header=2)
df_samples = df_samples.drop(['Cohort'], axis=1)
df_samples['Sample'] = df_samples.Sample.str.upper()
df_metadata = pd.merge(df_annotations, df_samples, left_on='sample_id', right_on='Sample')
df_metadata = df_metadata.drop(['Sample', 'treatment_group'], axis=1)
df_metadata.rename(columns={'Patient': 'patient_id',
                           'Age': 'age', 'Sex': 'sex', 'Lesion type': 'lesion_type', 'Site': 'site',
                           'Treatment': 'treatment', 'Treatment group': 'treatment_group'}, inplace=True)
#df_metadata.to_csv(METADATA_FNAME, index=False)
df_metadata.head()

In [ ]:
df_counts = pd.read_csv(EXP_MTX_COUNTS_FNAME,index_col=0,delimiter='\t')
df_counts.shape

In [ ]:
adata = sc.AnnData(X=df_counts.T.sort_index())
df_obs = df_metadata[['cell_id', 'sample_id', 'cell_type', 'cohort', 'patient_id', 'age', 'sex', 'treatment',
                                                           'treatment_group', 'lesion_type', 'site']].set_index('cell_id').sort_index()
adata.obs = df_obs
adata.var_names_make_unique()
adata

In [ ]:
adata.to_df().to_csv(EXP_MTX_QC_FNAME)

In [ ]:
!pyscenic grn {EXP_MTX_QC_FNAME} {HUMAN_TFS_FNAME} -o {ADJACENCIES_FNAME} --num_workers 4

In [ ]:
!pyscenic ctx {ADJACENCIES_FNAME} {DBS_PARAM} --annotations_fname {MOTIF_ANNOTATIONS_FNAME} --expression_mtx_fname {EXP_MTX_QC_FNAME} --output {MOTIFS_FNAME} --num_workers 26

In [ ]:
df_motifs = load_motifs(MOTIFS_FNAME)

In [ ]:
np.sum(df_motifs.index.isin(Selected_Genes,level='TF'))

In [ ]:
def derive_regulons(motifs, db_names=('hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings', 
                                 'hg19-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings', 
                                 'hg19-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings')):
    motifs.columns = motifs.columns.droplevel(0)

    def contains(*elems):
        def f(context):
            return any(elem in context for elem in elems)
        return f

    # For the creation of regulons we only keep the 10-species databases and the activating modules. We also remove the
    # enriched motifs for the modules that were created using the method 'weight>50.0%' (because these modules are not part
    # of the default settings of modules_from_adjacencies anymore.
    motifs = motifs[
        np.fromiter(map(compose(op.not_, contains('weight>50.0%')), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains(*db_names), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains('activating'), motifs.Context), dtype=np.bool)]

    # We build regulons only using enriched motifs with a NES of 3.0 or higher; we take only directly annotated TFs or TF annotated
    # for an orthologous gene into account; and we only keep regulons with at least 10 genes.
    regulons = list(filter(lambda r: len(r) >= 10, df2regulons(motifs[(motifs['NES'] >= 2.0)
                                                                      & ((motifs['Annotation'] == 'gene is directly annotated')
                                                                        | (motifs['Annotation'].str.startswith('gene is orthologous to')
                                                                           & motifs['Annotation'].str.endswith('which is directly annotated for motif')))
                                                                     ])))
    
    # Rename regulons, i.e. remove suffix.
    return list(map(lambda r: r.rename(r.transcription_factor), regulons))

In [ ]:
regulons = derive_regulons(df_motifs)

In [ ]:
with open(REGULONS_DAT_FNAME, 'wb') as f:
    pickle.dump(regulons, f)

auc_mtx = aucell(df_counts.T,regulons,num_workers=16) 
auc_mtx.to_csv(AUCELL_MTX_FNAME)

auc_mtx = pd.read_csv(AUCELL_MTX_FNAME, index_col=0)
tmpG = ['EOMES','MAFB','TBX21','MITF','MYC','PAX5','TCF7']
auc_for_bin = auc_mtx[tmpG]
auc_for_bin

bin_mtx, thresholds = binarize(auc_for_bin)
bin_mtx.to_csv(BIN_MTX_FNAME)
thresholds.to_frame().rename(columns={0:'threshold'}).to_csv(THR_FNAME)

bin_mtx = pd.read_csv(BIN_MTX_FNAME, index_col=0)
thresholds = pd.read_csv(THR_FNAME, index_col=0).threshold

add_scenic_metadata(adata, auc_mtx, regulons)

df_obs = adata.obs
signature_column_names = list(df_obs.select_dtypes('number').columns)
signature_column_names = list(filter(lambda s: s.startswith('Regulon('), signature_column_names))
df_scores = df_obs[signature_column_names + ['cell_type']]
df_results = ((df_scores.groupby(by='cell_type').mean() - df_obs[signature_column_names].mean())/ df_obs[signature_column_names].std()).stack().reset_index().rename(columns={'level_1': 'regulon', 0:'Z'})
df_results['regulon'] = list(map(lambda s: s[8:-1], df_results.regulon))
df_results[(df_results.Z >= 3.0)].sort_values('Z', ascending=False).head()
df_results.to_csv('test_SCENIC/ImputeEvaluation/results/GSE115978raw_CellTypeRegulonZ.csv')

In [ ]:
#afMF

In [ ]:
DATASET_ID = "GSE115978afMF"

CELL_ANNOTATIONS_FNAME = "test_SCENIC/ImputeEvaluation/GSE115978_cell.annotations.csv"

SAMPLE_METADATA_FNAME = os.path.join(RESOURCES_FOLDERNAME,"mmc1.xlsx")

EXP_MTX_COUNTS_FNAME = "afMF/imputed_data/SCENIC_GSE115978_sigma3_0_convergence_True.txt"

METADATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.metadata.csv'.format(DATASET_ID))
ADJACENCIES_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.adjacencies.tsv'.format(DATASET_ID))
MOTIFS_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.motifs.csv'.format(DATASET_ID))
REGULONS_DAT_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.regulons.dat'.format(DATASET_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.auc.csv'.format(DATASET_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.bin.csv'.format(DATASET_ID))
THR_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.thresholds.csv'.format(DATASET_ID))
ANNDATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.h5ad'.format(DATASET_ID))
EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.forSCENIC.csv'.format(DATASET_ID))

In [ ]:
df_annotations = pd.read_csv(CELL_ANNOTATIONS_FNAME)
df_annotations['samples'] = df_annotations['samples'].str.upper()
df_annotations.rename(columns={'cell.types': 'cell_type', 'cells': 'cell_id', 'samples': 'sample_id', 
                               'treatment.group': 'treatment_group', 'Cohort': 'cohort'}, inplace=True)
df_annotations['cell_type'] = df_annotations.cell_type.replace({
    'Mal': 'Malignant Cell',
    'Endo.': 'Endothelial Cell',
    'T.CD4': 'Thelper Cell',
    'CAF': 'Fibroblast',
    'T.CD8': 'Tcytotoxic Cell',
    'T.cell': 'T Cell',
    'NK': 'NK Cell',
    'B.cell': 'B Cell'})
df_samples = pd.read_excel(SAMPLE_METADATA_FNAME, header=2)
df_samples = df_samples.drop(['Cohort'], axis=1)
df_samples['Sample'] = df_samples.Sample.str.upper()
df_metadata = pd.merge(df_annotations, df_samples, left_on='sample_id', right_on='Sample')
df_metadata = df_metadata.drop(['Sample', 'treatment_group'], axis=1)
df_metadata.rename(columns={'Patient': 'patient_id',
                           'Age': 'age', 'Sex': 'sex', 'Lesion type': 'lesion_type', 'Site': 'site',
                           'Treatment': 'treatment', 'Treatment group': 'treatment_group'}, inplace=True)
#df_metadata.to_csv(METADATA_FNAME, index=False)
df_metadata.head()

In [ ]:
df_counts = pd.read_csv(EXP_MTX_COUNTS_FNAME,index_col=0,delimiter='\t')
print(df_counts.shape)

adata = sc.AnnData(X=df_counts.T.sort_index())
df_obs = df_metadata[['cell_id', 'sample_id', 'cell_type', 'cohort', 'patient_id', 'age', 'sex', 'treatment',
                                                           'treatment_group', 'lesion_type', 'site']].set_index('cell_id').sort_index()
adata.obs = df_obs
adata.var_names_make_unique()
print(adata)

adata.to_df().to_csv(EXP_MTX_QC_FNAME)

In [ ]:
#batch job

#!pyscenic grn {EXP_MTX_QC_FNAME} {HUMAN_TFS_FNAME} -o {ADJACENCIES_FNAME} --num_workers 4
#!pyscenic ctx {ADJACENCIES_FNAME} {DBS_PARAM} --annotations_fname {MOTIF_ANNOTATIONS_FNAME} --expression_mtx_fname {EXP_MTX_QC_FNAME} --output {MOTIFS_FNAME} --num_workers 26

In [ ]:
df_motifs = load_motifs(MOTIFS_FNAME)
def derive_regulons(motifs, db_names=('hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings', 
                                 'hg19-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings', 
                                 'hg19-tss-centered-5kb-10species.mc9nr.genes_vs_motifs.rankings')):
    motifs.columns = motifs.columns.droplevel(0)

    def contains(*elems):
        def f(context):
            return any(elem in context for elem in elems)
        return f

    # For the creation of regulons we only keep the 10-species databases and the activating modules. We also remove the
    # enriched motifs for the modules that were created using the method 'weight>50.0%' (because these modules are not part
    # of the default settings of modules_from_adjacencies anymore.
    motifs = motifs[
        np.fromiter(map(compose(op.not_, contains('weight>50.0%')), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains(*db_names), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains('activating'), motifs.Context), dtype=np.bool)]

    # We build regulons only using enriched motifs with a NES of 3.0 or higher; we take only directly annotated TFs or TF annotated
    # for an orthologous gene into account; and we only keep regulons with at least 10 genes.
    regulons = list(filter(lambda r: len(r) >= 10, df2regulons(motifs[(motifs['NES'] >= 2.0)
                                                                      & ((motifs['Annotation'] == 'gene is directly annotated')
                                                                        | (motifs['Annotation'].str.startswith('gene is orthologous to')
                                                                           & motifs['Annotation'].str.endswith('which is directly annotated for motif')))
                                                                     ])))
    
    # Rename regulons, i.e. remove suffix.
    return list(map(lambda r: r.rename(r.transcription_factor), regulons))
regulons = derive_regulons(df_motifs)

In [ ]:
tmp = [x.name for x in regulons]
tmp = [x in tmp for x in ['EOMES','MAFB','TBX21','MITF','MYC','PAX5','TCF7']]
tmp

In [ ]:
with open(REGULONS_DAT_FNAME, 'wb') as f:
    pickle.dump(regulons, f)

auc_mtx = aucell(df_counts.T,regulons,num_workers=16) 
auc_mtx.to_csv(AUCELL_MTX_FNAME)

auc_mtx = pd.read_csv(AUCELL_MTX_FNAME, index_col=0)
tmpG = ['EOMES','MAFB','TBX21','MITF','MYC','PAX5','TCF7']
auc_for_bin = auc_mtx[tmpG]
print(auc_for_bin)

bin_mtx, thresholds = binarize(auc_for_bin)
bin_mtx.to_csv(BIN_MTX_FNAME)
thresholds.to_frame().rename(columns={0:'threshold'}).to_csv(THR_FNAME)

bin_mtx = pd.read_csv(BIN_MTX_FNAME, index_col=0)
thresholds = pd.read_csv(THR_FNAME, index_col=0).threshold

In [ ]:
N_COLORS = len(adata.obs.cell_type.unique())
COLORS = [color['color'] for color in mpl.rcParams["axes.prop_cycle"]]

sns.set()
sc.pp.highly_variable_genes(adata)
sc.pl.highly_variable_genes(adata)
#adata = adata[:, adata.var['highly_variable']]

sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca(adata, color='CD8A')

sc.tl.tsne(adata)
sc.set_figure_params(frameon=False, dpi=150, fontsize=8)
sc.pl.tsne(adata, color=['cell_type', 'lesion_type', 'patient_id'], 
           title=['GSE115978 - SKCM - Cell types', 'GSE115978 - SKCM - Lesion types',
                 'GSE115978 - SKCM - {} patients'.format(len(adata.obs.patient_id.unique()))], ncols=3, palette=COLORS)

add_scenic_metadata(adata, auc_mtx, regulons)
df_obs = adata.obs
signature_column_names = list(df_obs.select_dtypes('number').columns)
signature_column_names = list(filter(lambda s: s.startswith('Regulon('), signature_column_names))
df_scores = df_obs[signature_column_names + ['cell_type']]
df_results = ((df_scores.groupby(by='cell_type').mean() - df_obs[signature_column_names].mean())/ df_obs[signature_column_names].std()).stack().reset_index().rename(columns={'level_1': 'regulon', 0:'Z'})
df_results['regulon'] = list(map(lambda s: s[8:-1], df_results.regulon))
print(df_results[(df_results.Z >= 3.0)].sort_values('Z', ascending=False).head())

print(np.sum(df_results.Z >= 3.0))
print(np.sum(df_results.Z >= 2.0))

df_results.to_csv('test_SCENIC/ImputeEvaluation/results/GSE115978scRMD_CellTypeRegulonZ.csv')

